In [ ]:
%pip install -r TrajVis/requirements.txt

In [9]:
from TrajVis.components.balloon import Balloon
from TrajVis.components.network import Network
from TrajVis.components.networkanalyzer import NetworkAnalyzer as na
from TrajVis.components.plothelper import PlotHelper as ph
from TrajVis.components.windmap import WindMap as wm
from TrajVis.components.sample_antennas import *
from TrajVis.components.datahandler import DataHandler as dh
from antennas import *
import copy

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [10]:
MyWindMap = wm(500, 500, 1)
MyWindMap.create_windmap()
MyWindMap.bound(4)

MyPlotHelper = ph(MyWindMap)
MyPlotHelper.populate(show_scale=False)
# MyPlotHelper.plot(debug=False)

B1 = Balloon(MyWindMap, "1", "red")
B2 = Balloon(MyWindMap, "2", "blue")
B3 = Balloon(MyWindMap, "3", "green")
B4 = Balloon(MyWindMap, "4", "orange")

B1.set_start_pos(-20, -45)
B2.set_start_pos(-20, -48)
B3.set_start_pos(-21, -33)
B4.set_start_pos(-20, -38)

tickResponse_1 = 1
tickResponse_2 = 1
tickResponse_3 = 1
tickResponse_4 = 1

AntennaToUse = CPXband
B1.add_antenna(AntennaToUse)
B2.add_antenna(AntennaToUse)
B3.add_antenna(AntennaToUse)
B4.add_antenna(AntennaToUse)

# MyPlotHelper.plot_balloons(B1, B2, B3, B4, B5)
# MyPlotHelper.zoom_in(100, 100)

NA = na(B1, B2, B3, B4)

MyNetwork = Network(B1,
                    B2,
                    B3,
                    B4,
                    start=B1,
                    end=B4,
                    generic_path=["1", "2", "3", "4"])

while tickResponse_1:
    tickResponse_1 = B1.tick()
while tickResponse_2:
    tickResponse_2 = B2.tick()
while tickResponse_3:
    tickResponse_3 = B3.tick()
while tickResponse_4:
    tickResponse_4 = B4.tick()

1  popped at  7738.0
1  hit the ground at time t= 10003.0
2  popped at  7122.5
2  hit the ground at time t= 9389.0
3  popped at  6770.0
3  hit the ground at time t= 9036.5
4  popped at  6673.5
4  hit the ground at time t= 8940.5


# Generating Coords from APRS

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

In [12]:
# hijack the balloons with actual APRS data
def latlng_to_xy(lat, lng, origin_lat, origin_lng):
    """
    Convert latitude and longitude to relative X, Y coordinates in meters.
    Uses a simple equirectangular projection approximation.
    """
    R = 6371000  # Earth radius in meters
    x = R * np.radians(lng - origin_lng) * np.cos(np.radians(origin_lat))
    y = R * np.radians(lat - origin_lat)
    return x, y

def linear_interpolate(x, y, target_x):
    """
    Perform linear interpolation or edge value extrapolation.
    x: Array of existing timestamps
    y: Array of values corresponding to x (same length)
    target_x: Target timestamp to interpolate/extrapolate
    """
    if target_x <= x[0]:
        return y[0]  # Use the first value for timestamps before the range
    elif target_x >= x[-1]:
        return y[-1]  # Use the last value for timestamps after the range
    else:
        idx = np.searchsorted(x, target_x)  # Find the index of the closest larger value
        x0, x1 = x[idx - 1], x[idx]
        y0, y1 = y[idx - 1], y[idx]
        return y0 + (y1 - y0) * (target_x - x0) / (x1 - x0)  # Linear interpolation formula

def process_balloon_data(folder_path):
    data = {}
    all_timestamps = []
    origin_lat, origin_lng = None, None
    
    # Step 1: Parse all files and collect timestamps
    files_data = {}
    for file_name in sorted(os.listdir(folder_path)):
        if file_name.endswith('.csv'):
            team_name = file_name.split()[0]  # Extract team name
            file_path = os.path.join(folder_path, file_name)
            
            # Load data
            df = pd.read_csv(file_path)
            df['timestamp'] = pd.to_datetime(df['time']).astype('int64') // 10**9  # Convert to UNIX time
            
            # Initialize origin_lat, origin_lng from the first file's first row
            if origin_lat is None or origin_lng is None:
                origin_lat, origin_lng = df['lat'][0], df['lng'][0]
            
            # Convert lat/lng to relative x, y
            df['x'], df['y'] = zip(*df.apply(lambda row: latlng_to_xy(row['lat'], row['lng'], origin_lat, origin_lng), axis=1))
            
            files_data[team_name] = df
            all_timestamps.extend(df['timestamp'].tolist())
    
    # Step 2: Determine the unified timestamp range
    earliest_timestamp = min(all_timestamps)
    latest_timestamp = max(all_timestamps)
    
    # Extend the range to cover an integer number of minutes
    earliest_timestamp = earliest_timestamp - (earliest_timestamp % 60)
    latest_timestamp = latest_timestamp + (60 - latest_timestamp % 60)
    unified_timestamps = np.arange(earliest_timestamp, latest_timestamp + 1, 60)  # 1-minute steps
    
    # Step 3: Interpolate data for each team
    for team_name, df in files_data.items():
        timestamps = df['timestamp'].values
        x_values = df['x'].values
        y_values = df['y'].values
        z_values = df['altitude'].values
        
        # Interpolate x, y, z for unified timestamps
        interpolated_x = [linear_interpolate(timestamps, x_values, t) for t in unified_timestamps]
        interpolated_y = [linear_interpolate(timestamps, y_values, t) for t in unified_timestamps]
        interpolated_z = [linear_interpolate(timestamps, z_values, t) for t in unified_timestamps]
        
        # Combine into 1D array of [x, y, z]
        data[team_name] = np.column_stack((interpolated_x, interpolated_y, interpolated_z))
    
    return data, unified_timestamps

folder_path = "./f23_495_aprs"
balloon_data, unified_timestamps = process_balloon_data(folder_path)

# Output example
for team, coordinates in balloon_data.items():
    print(f"Team {team}: {len(coordinates)} coordinates")

Team Mike: 89 coordinates
Team November: 89 coordinates
Team Papa: 89 coordinates
Team Quebec: 89 coordinates


In [ ]:
# hijack balloons with coords

for team, coords in balloon_data.items():
    if team == "Mike":
        B1.coordinates = coords
        B1.attitudes = B1.attitudes[:len(coords)]
    elif team == "November":
        B2.coordinates = coords
        B2.attitudes = B2.attitudes[:len(coords)]
    elif team == "Papa":
        B3.coordinates = coords
        B3.attitudes = B3.attitudes[:len(coords)]
    elif team == "Quebec":
        B4.coordinates = coords
        B4.attitudes = B4.attitudes[:len(coords)]

In [ ]:
reconfig_rp = []
generic_rp = []
reconfig_sensitivity = []
generic_sensitivity = []
ge_sensitivities_time = []
for i in range(1, 90):
    MyNetwork.recalculate(i, NA.rp)
    MyNetwork.transmit(i, NA.rp)
    
    re_total_rp, re_sensitivity_test = MyNetwork.calculate_total_rp(i, NA.rp, path="Current")
    generic_total_rp, ge_sensitivity_test = MyNetwork.calculate_total_rp(i, NA.rp, path="Generic")
    
    reconfig_rp.append(re_total_rp)
    generic_rp.append(generic_total_rp)
    
    reconfig_sensitivity.append(re_sensitivity_test)
    if ge_sensitivity_test:
        generic_sensitivity.append(generic_total_rp)
        ge_sensitivities_time.append(i)
    
# NA.show_pointing(3500, B1, B2, B3, B4, B5, B6, B7, B8)

KeyError: 'Mike'

In [ ]:
myDH = dh(MyNetwork)

print(len(reconfig_rp), len(generic_rp))

# myDH.trp_comparison_to_csv(path1=reconfig_rp, path2=generic_rp, generic_sensitivities=[ge_sensitivities_time, generic_sensitivity], path1_name="Reconfigurable Path", path2_name="Generic Path", plot_meaningfuls=True, system="A")

myDH.plot_trp_comparison(path1=reconfig_rp, path2=generic_rp, generic_sensitivities=[ge_sensitivities_time, generic_sensitivity], path1_name="Reconfigurable Path", path2_name="Generic Path", plot_meaningfuls=True, system="B")